In [3]:
from IPython.display import Latex
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Basics.DataStructures import Vector
import numpy as np
import os,shutil,glob
path_dir = os.getcwd()
caso_referencia = path_dir + "/alaBatiente"
work_dir = os.getcwd()

\begin{equation}
\omega_n = \left(\frac{kL}{L}\right)^2 \sqrt{\frac{EI}{\mu}} = \left(\frac{kL}{L}\right)^2 \sqrt{\frac{EI/b}{\mu/b}}
\end{equation}
$$m = \rho L e b$$

$$\mu = m/L$$

In [4]:
U_s = [0.80, 1.00]
f_fs = [12.00, 14.]
L =35e-3
rho = 1623
E =5.62547e09
sigma_p = 0.38
espesor = 50e-6
b =160 # envergadura
mu_lineal = rho*espesor
kL = 1.875
I_b = espesor**3/12
wn = (kL/L)**2*np.sqrt(E*I_b/mu_lineal)
fn = wn/2/np.pi
display(Latex(f'frecuencia natural: {fn:.2f} Hz'))
theta0 = 0.1745329
display(Latex(f'Amplitud angular: {theta0*180/np.pi:.2f}$^\circ$ '))

i = 0; j = 0
f_fj = f_fs[j] / fn
U_Ri = U_s[i] / (wn*L)

display(Latex(f'Velocidad reducida: $U_R={U_Ri:.2f}$'))
display(Latex(f'Frecuencia adimensional: $f^+={f_fj:.2f}$ '))
path_caso = path_dir + f"/UR{U_Ri:.2f}_f{f_fj:.2f}"

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
if not os.path.isdir(path_caso):
    shutil.copytree(caso_referencia,path_caso)
path_caso_fluido = path_caso+'/fluid'
caso_foam = SolutionDirectory(path_caso_fluido, archive=None, paraviewLink=False)  
#para caso foam se pueden usar las funciones de pyfoam, pej cambiar condiciones de borde como velocidad



In [11]:
# SimpleFoam

# Aplica Velocidad
file_U_txt = path_caso+'/simpleFoam/0/U'
with open(file_U_txt, "r+") as text_file:
    texts = text_file.read()
    texts = texts.replace("0.78", f"{U_Ri*wn*L:.2f}")
with open(file_U_txt, "w") as text_file:
    text_file.write(texts)

#Ejecuta
os.chdir(path_caso+'/simpleFoam')
os.system('simpleFoam')
os.chdir(work_dir)

In [51]:
0#selecciona ultima solucion
lista_files_0 = glob.glob(path_caso+'/simpleFoam/*')
lista_files_1 = []
indices = []
for filei in lista_files_0:
    string1 = filei.split('/')[-1] 
    if any(c.isdigit() for c in string1):
        lista_files_1.append(filei)
        indices.append(int(string1))
indices = np.asarray(indices)        
lista_files_1 = np.asarray(lista_files_1)[indices.argsort()]
        

In [54]:
#Copia el ultimo resultado a fluid
    
lista_files = glob.glob(lista_files_1[-1]+'/*')
for filei in lista_files:
    if not os.path.isdir(filei):
        shutil.copy(filei,path_caso+'/fluid/0/')

In [26]:
#parametros ccx. En solido/ala.inp / precice config / uboun.f

omega = np.pi*10
r = 10e-4  #radio del eje que oscila 
ang1 = np.arcsin(25e-5 / r)
display(Latex(f'Amplitud angular: {ang1*180/np.pi:.2f}$^\circ$ '))

<IPython.core.display.Latex object>

[Ver](https://jhoepken.github.io/blog/2012/01/22/parameter-variation-in-openfoam-using-pyfoam/) para un ejemplo de variación de condiciones de borde con pyfoam.

De todas formas, hay que terminar editando con comandos como los anteriores para modificar ccx y detalles del OpenFoam (tiempos)